In [67]:
import json

resumes = []
with open('Entity Recognition in Resumes.jsonl') as f:
    for line in f:
        resume = json.loads(line)
        resumes.append(resume)

In [68]:


resume_index= 1
resume = resumes[resume_index]

import os
file_path = f"data/evaluation/json/resume_{resume_index}.json"
if os.path.exists(file_path):
    print('the file already exists')

In [69]:
prompt = ''

prompt += '''
extract jobs positions along with attributes of the position: title, company, start_year, end_year
job positions only, not projects.

example output
[
    {"position_title": "aaa", "company":"bbb", "start_year": null, "end_year": 2023},
    {"position_title": "aaa", "company":"bbb", "start_year": 2021, "end_year": null}
]


'''

prompt = """
below is extract annotation from a resume      
====

"""

for item in resume['annotation']:
    prompt += f"=== {item['label']=} ===\n"
    for point in item['points']:
        prompt += f"{point['text']}\n"
    prompt += "\n"

print(prompt)


below is extract annotation from a resume      
====

=== item['label']=['Skills'] ===
Teaching

=== item['label']=['College Name'] ===
R.D.H.S. School

=== item['label']=['Degree'] ===
Post Graduate Diploma in Clinical Data Management

=== item['label']=['College Name'] ===
R.D.H.S. School

=== item['label']=['Degree'] ===
H.S.C

=== item['label']=['College Name'] ===
Government Science College

=== item['label']=['Degree'] ===
B. Sc. in Biology

=== item['label']=['Companies worked at'] ===
Oracle Clinical

=== item['label']=['Designation'] ===
Class teacher

=== item['label']=['Location'] ===
Pune

=== item['label']=['Designation'] ===
Preschool Teacher

=== item['label']=['Email Address'] ===
indeed.com/r/Bhawana-Daf/d9ddb6a54519d583


=== item['label']=['Location'] ===
Pune

=== item['label']=['Name'] ===
Bhawana Daf




In [70]:
from src.llm.lmstudio import do_llm_completion

messages = [
    {
        "role": "system",
        "content": """
extract jobs positions along with attributes of the position: title, company, start_year, end_year
job positions only, not projects.

example output
[
    {"position_title": "aaa", "company":"bbb", "start_year": null, "end_year": 2023},
    {"position_title": "aaa", "company":"bbb", "start_year": 2021, "end_year": null}
]
"""
    },
    {
        "role": "user",
        "content": prompt,
    }
]

llm_response = do_llm_completion(messages)

print('raw output')
print(llm_response)

print('clean output')
from src.utils.extract_json import extract_json_only
response_json = extract_json_only(llm_response)
print(json.dumps(json.loads(response_json[0]), indent=4))


raw output
Here is the extracted information in a JSON format:

```
[
    {
        "position_title": null,
        "company": null,
        "start_year": null,
        "end_year": null
    },
    {
        "position_title": "Class teacher",
        "company": "Oracle Clinical",
        "start_year": null,
        "end_year": null
    },
    {
        "position_title": "Preschool Teacher",
        "company": null,
        "start_year": null,
        "end_year": null
    }
]
```

Note that the first two entries are not actual job positions, but rather educational qualifications. The third entry is a job position with no start or end year specified.
clean output
[
    {
        "position_title": null,
        "company": null,
        "start_year": null,
        "end_year": null
    },
    {
        "position_title": "Class teacher",
        "company": "Oracle Clinical",
        "start_year": null,
        "end_year": null
    },
    {
        "position_title": "Preschool Teacher",
      

In [71]:
from src.llm.lmstudio import do_llm_completion

messages = [
    {
        "role": "system",
        "content": """
you are resume parser.
extract email address of this person from his resume.
output in json

example output
[
    { "value": "xxx@gmail.com" }
]

if there is no e-mail in his resume. output a blank array like this
[ ]
"""
    },
    {
        "role": "user",
        "content": prompt,
    }
]

llm_response = do_llm_completion(messages)

print('raw output')
print(llm_response)

print('clean output')
from src.utils.extract_json import extract_json_only
response_json = extract_json_only(llm_response)
print(json.dumps(json.loads(response_json[0]), indent=4))

extracted_emails = []
if len(response_json) > 0:
    extracted_emails = json.loads(response_json[0])

raw output
Based on the extracted annotation from the resume, I can extract the email address as follows:

```
[
    {"value": "indeed.com/r/Bhawana-Daf/d9ddb6a54519d583"}
]
```

Note that this is a URL-style email address, which may not be in the traditional format of `username@domain.com`.
clean output
[
    {
        "value": "indeed.com/r/Bhawana-Daf/d9ddb6a54519d583"
    }
]


In [72]:
from src.llm.lmstudio import do_llm_completion

messages = [
    {
        "role": "system",
        "content": """
you are resume parser.
extract phone number of this person from his resume.
output in json

example output
[{ "type": "Telephone", "value": "4204208484" }]

if there is no phone number in his resume. output a blank array like this
[ ]
"""
    },
    {
        "role": "user",
        "content": prompt,
    }
]

llm_response = do_llm_completion(messages)

print('raw output')
print(llm_response)

print('clean output')
from src.utils.extract_json import extract_json_only
response_json = extract_json_only(llm_response)
print(response_json)

extracted_phones = []
if len(response_json) > 0:
    extracted_emails = json.loads(response_json[0])

raw output
I can extract the phone number from this resume. However, I don't see any phone number mentioned in this annotation. Therefore, my output will be a blank array:

[ ]

If you could provide me with the actual resume text or a more detailed annotation that includes the phone number, I'll be happy to help you extract it and output it in JSON format as requested.
clean output
[]


In [73]:
from src.llm.lmstudio import do_llm_completion

messages = [
    {
        "role": "system",
        "content": """
you are resume parser.
extract his job positions and information related to each position.
output in json

example output
[
    {
      "city": "",
      "title": "NCAT ASEC",
      "country": "",
      "employer": "Hickman Property Holdings LLC",
      "end_date": "2023-",
      "start_date": "2022-2",
      "country_code": ""
    }
]

if there is no job position in his resume. output a blank array like this
[ ]
"""
    },
    {
        "role": "user",
        "content": prompt,
    }
]

llm_response = do_llm_completion(messages)

print('raw output')
print(llm_response)

print('clean output')
from src.utils.extract_json import extract_json_only
response_json = extract_json_only(llm_response)
print(response_json)
print(json.dumps(json.loads(response_json[0]), indent=4))

extracted_jobs = []
if len(response_json) > 0:
    extracted_jobs = json.loads(response_json[0])

raw output
Based on the provided annotation, I can extract the job positions and related information. Here is the output in JSON format:

[
    {
        "city": "Pune",
        "title": "Class Teacher",
        "country": "",
        "employer": "Oracle Clinical",
        "end_date": "",
        "start_date": "",
        "country_code": ""
    },
    {
        "city": "Pune",
        "title": "Preschool Teacher",
        "country": "",
        "employer": "",
        "end_date": "",
        "start_date": "",
        "country_code": ""
    }
]

Note that there is no specific start and end date mentioned for the job positions, so I have left those fields blank. Also, the employer for the Preschool Teacher position is not specified, so it's left as an empty string.
clean output
['[\n    {\n        "city": "Pune",\n        "title": "Class Teacher",\n        "country": "",\n        "employer": "Oracle Clinical",\n        "end_date": "",\n        "start_date": "",\n        "country_code": "

In [74]:
from src.llm.lmstudio import do_llm_completion

messages = [
    {
        "role": "system",
        "content": """
you are resume parser.
extract his skills as specified in his resume. both hard and soft skills.
output in json

example output
[
    {"name": "Community Service"},
    {"name": "Concord"}
]

if there is no skills in his resume, output a blank array like this
[ ]
"""
    },
    {
        "role": "user",
        "content": prompt,
    }
]

llm_response = do_llm_completion(messages)

print('raw output')
print(llm_response)

print('clean output')
from src.utils.extract_json import extract_json_only
response_json = extract_json_only(llm_response)
print(response_json)
print(json.dumps(json.loads(response_json[0]), indent=4))

extracted_skills = []
if len(response_json) > 0:
    extracted_skills = json.loads(response_json[0])

raw output
Based on the provided annotation, I will extract the skills mentioned in the resume. Since there are no explicit "Skills" sections or keywords listed, I will focus on extracting relevant information that can be interpreted as skills.

Here is the extracted skill set in JSON format:

```
[
  {"name": "Teaching"},
  {"name": "Clinical Data Management"},
  {"name": "Biology"}
]
```

Note that these skills are inferred based on the education and work experience mentioned in the resume. The skills may not be exhaustive or directly listed, but they can be interpreted as relevant to the candidate's profession.

If there were no skills mentioned in the resume, the output would be an empty array:

```
[]
```
clean output
['[\n  {"name": "Teaching"},\n  {"name": "Clinical Data Management"},\n  {"name": "Biology"}\n]']
[
    {
        "name": "Teaching"
    },
    {
        "name": "Clinical Data Management"
    },
    {
        "name": "Biology"
    }
]


In [75]:
from src.llm.lmstudio import do_llm_completion

messages = [
    {
        "role": "system",
        "content": """
you are resume parser.
extract his education as specified in his resume.
output in json

example output
[
  {
    "city": "",
    "school": "State University",
    "country": "",
    "end_date": "2024-5",
    "start_date": "-",
    "degree_name": "",
    "description": "",
    "country_code": "NC",
    "degree_major": ""
  },
  {
    "city": "",
    "school": "Central Piedmont Community College",
    "country": "",
    "end_date": "2021-12",
    "start_date": "2020-8",
    "degree_name": "",
    "description": "North Carolina Agricultural and Technical State University, Greensboro, NC\nMay 2024\nGPA: 3.4\nCentral Piedmont Community College, Charlotte NC\nAugust 2020 - December 2021\nGPA: 3.3\nCourse Careers, Remote December 2022 - January 2023\nGPA: 3.3\nCourse Careers, Remote December 2022 - January 2023",
    "country_code": "NC",
    "degree_major": ""
  }
]

if there is no education in his resume, output a blank array like this
[ ]
"""
    },
    {
        "role": "user",
        "content": prompt,
    }
]

llm_response = do_llm_completion(messages)

print('raw output')
print(llm_response)

print('clean output')
from src.utils.extract_json import extract_json_only
response_json = extract_json_only(llm_response)
print(response_json)
print(json.dumps(json.loads(response_json[0]), indent=4))

extracted_educations = []
if len(response_json) > 0:
    extracted_educations = json.loads(response_json[0])


raw output
Based on the extracted annotation from the resume, here is the education information in JSON format:

```
[
  {
    "city": "",
    "school": "R. D. H. S. School",
    "country": "",
    "end_date": "",
    "start_date": "",
    "degree_name": "Post Graduate Diploma in Clinical Data Management",
    "description": "",
    "country_code": "",
    "degree_major": ""
  },
  {
    "city": "",
    "school": "Government Science College",
    "country": "",
    "end_date": "",
    "start_date": "",
    "degree_name": "B. Sc. in Biology",
    "description": "",
    "country_code": "",
    "degree_major": ""
  }
]
```

Note that the `end_date` and `start_date` fields are empty since they were not provided in the original annotation.
clean output
['[\n  {\n    "city": "",\n    "school": "R. D. H. S. School",\n    "country": "",\n    "end_date": "",\n    "start_date": "",\n    "degree_name": "Post Graduate Diploma in Clinical Data Management",\n    "description": "",\n    "country_code

In [76]:
output_data = {
	"personal": {
    "gender": "",
    "full_name": "Hidden",
    "birthplace": "",
    "first_name": "Hidden",
    "family_name": "Hidden",
    "middle_name": "Hidden",
    "nationality": [],
    "picture_url": None,
    "date_of_birth": "",
    "marital_status": "",
    "picture_extension": None
  },
  "contact": {
    "email": [{ "value": "xxx@gmail.com" }],
    "phone": [{ "type": "Telephone", "value": "4204208484" }],
    "address": [],
    "website": []
  },
  "summary": {
    "benefits": "",
    "objective": "",
    "description": "",
    "notice_period": "",
    "current_salary": ""
  },
  "metadata": {
    "job_pk": 463152,
    "remark": "Parsing Complete",
    "status": "succeeded",
    "resume_pk": 15801024,
    "candidate_pk": 21380189,
    "language_code": "en",
    "language_confidence": 0.9999951854043442,
  },
  "education": [
    {
      "city": "",
      "school": "State University",
      "country": "",
      "end_date": "2024-5",
      "start_date": "-",
      "degree_name": "",
      "description": "",
      "country_code": "NC",
      "degree_major": "",
      "custom_sections": []
    },
    {
      "city": "",
      "school": "Central Piedmont Community College",
      "country": "",
      "end_date": "2021-12",
      "start_date": "2020-8",
      "degree_name": "",
      "description": "North Carolina Agricultural and Technical State University, Greensboro, NC\nMay 2024\nGPA: 3.4\nCentral Piedmont Community College, Charlotte NC\nAugust 2020 - December 2021\nGPA: 3.3\nCourse Careers, Remote December 2022 - January 2023\nGPA: 3.3\nCourse Careers, Remote December 2022 - January 2023",
      "country_code": "NC",
      "degree_major": "",
      "custom_sections": []
    }
  ],
  
  "experience": [
    {
      "city": "",
      "title": "NCAT ASEC",
      "country": "",
      "employer": "Hickman Property Holdings LLC",
      "end_date": "2023-",
      "start_date": "2022-2",
      "description": "● A group of like-minded individuals who also are involved on and off campus community service activities",
      "country_code": "",
      "custom_sections": []
    },
    {
      "city": "",
      "title": "Founder",
      "country": "",
      "employer": "Hickman Property Holdings LLC",
      "end_date": "2023-",
      "start_date": "2022-9",
      "description": "● I have 6 employees that I manage and work within my company.",
      "country_code": "",
      "custom_sections": []
    },
    {
      "city": "",
      "title": "Merchandiser",
      "country": "",
      "employer": "Fedex",
      "end_date": "2022-",
      "start_date": "2021-",
      "description": "● Provided great customer service by being knowledgeable about all the consu(...)",
      "country_code": "NC",
      "custom_sections": []
    },
    {
      "city": "",
      "title": "Retail Sales Specialist",
      "country": "",
      "employer": "Fedex",
      "end_date": "2022-",
      "start_date": "2022-",
      "description": "● Provide the Best Customer Service\nProvide the best sales experience(...)",
      "country_code": "",
      "custom_sections": []
    },
    {
      "city": "",
      "title": "Team Lead",
      "country": "",
      "employer": "Burger King",
      "end_date": "2020-",
      "start_date": "2018-",
      "description": "● Utilize leadership skills to properly train new employees",
      "country_code": "NC",
      "custom_sections": []
    }
  ],
	"languages": [{ "code": "en", "name": "English", "description": "I am fluent - Level C1"}],
  "skills": [
    "Community Service",
    "Concord",
    "FedEx",
    "Hard Labor",
    "Leadership",
    "Textbooks",
    "Toys"
  ],
  "achievements": {},
  "certifications": {},
  "qualifications": {}
}

In [77]:
output_data['skills'] = [item['name'] for item in extracted_skills]
output_data['education'] = extracted_educations
output_data['experience'] = extracted_jobs
output_data['contact']['email'] = extracted_emails
output_data['contact']['phone'] = extracted_phones

In [78]:
print(json.dumps(output_data, indent=4))

{
    "personal": {
        "gender": "",
        "full_name": "Hidden",
        "birthplace": "",
        "first_name": "Hidden",
        "family_name": "Hidden",
        "middle_name": "Hidden",
        "nationality": [],
        "picture_url": null,
        "date_of_birth": "",
        "marital_status": "",
        "picture_extension": null
    },
    "contact": {
        "email": [
            {
                "value": "indeed.com/r/Bhawana-Daf/d9ddb6a54519d583"
            }
        ],
        "phone": [],
        "address": [],
        "website": []
    },
    "summary": {
        "benefits": "",
        "objective": "",
        "description": "",
        "notice_period": "",
        "current_salary": ""
    },
    "metadata": {
        "job_pk": 463152,
        "remark": "Parsing Complete",
        "status": "succeeded",
        "resume_pk": 15801024,
        "candidate_pk": 21380189,
        "language_code": "en",
        "language_confidence": 0.9999951854043442
    },
    "e

In [79]:
file_path = f"data/evaluation/json/resume_{resume_index}.json"
print('write to file:', file_path)
with open(file_path, 'w') as f:
    f.write(json.dumps(output_data, indent=4))

write to file: data/evaluation/json/resume_14.json
